In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
runs = {'v2.1':   'run144',
        'v2.2.1 (growth summary updates)': 'run325',
        'v2.3':   'run241',
        'v2.3.1 (devproj updates)': 'run242',
        'v2.4':   'run152',
        'v2.5':   'run327',
        'v2.6':   'run243',
        'v2.7':   'run158',
        'v2.8':   'run335',
        'v2.9':   'run160'}

if os.getenv('USERNAME')=='ywang':
    input_folder = 'C:\\Users\\{}\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Final Blueprint runs\\Final Blueprint (s24)'.format(os.getenv('USERNAME'))
    working_dir = 'M:\\Data\\Urban\\BAUS\\PBA50\\Final_Blueprint\\run_analysis\\SantaClara_county_subsidized_strategy'

os.chdir(working_dir)

def version_name(x):
    if '(' in x:
        return x.split(' ')[0]
    else:
        return x

In [3]:
parcel_geo = pd.read_csv(r'M:\Data\Urban\BAUS\PBA50\Final_Blueprint\Large General Input Data\2020_10_27_parcels_geography.csv',
                         usecols = ['PARCEL_ID', 'juris'])

juris_county = pd.read_csv(os.path.join('util','juris_county_id.csv'),
                           usecols = ['juris_name_full', 'county_name'])

parcel_geo = parcel_geo.merge(juris_county, left_on='juris', right_on='juris_name_full', how='left')
parcel_geo_scl = parcel_geo.loc[parcel_geo.county_name == 'Santa Clara']
print('Santa Clara County has {} parcels'.format(parcel_geo_scl.shape[0]))

display(parcel_geo_scl)

Santa Clara County has 436449 parcels


,PARCEL_ID,juris,juris_name_full,county_name
143,1440230,sunnyvale,sunnyvale,Santa Clara
483,1412789,san_jose,san_jose,Santa Clara
538,1408883,san_jose,san_jose,Santa Clara
549,1309312,palo_alto,palo_alto,Santa Clara
583,1648785,san_jose,san_jose,Santa Clara
...,...,...,...,...
1956189,1417339,san_jose,san_jose,Santa Clara
1956190,1417349,san_jose,san_jose,Santa Clara
1956191,1417392,san_jose,san_jose,Santa Clara
1956192,1417403,san_jose,san_jose,Santa Clara


In [4]:
for i in runs:
    print(i)
    
    blg_file = input_folder+'\\'+'BAUS '+i+'\\'+runs[i]+'_building_data_2050.csv'
    acct_file = input_folder+'\\'+'BAUS '+i+'\\'+runs[i]+'_acctlog_Santa Clara Affordable Housing Bond_2050.csv'
    blg = pd.read_csv(blg_file)
    blg['source_detail'] = blg['source']
    blg = blg.merge(parcel_geo[['PARCEL_ID', 'juris', 'county_name']],
                    left_on='parcel_id',
                    right_on='PARCEL_ID',
                    how='left')
    blg_scl = blg.loc[blg.county_name == 'Santa Clara']
    acct = pd.read_csv(acct_file)
    #display(acct.loc[acct.building_id.notnull()])

    acct.columns = [x+'_acct' for x in list(acct)]
    
    if 'building_id_acct' in acct:
        
        blg_scl_join = blg_scl.merge(acct, 
                                     left_on = 'building_id', 
                                     right_on = 'building_id_acct', 
                                     how = 'left')
        
        blg_scl_join.loc[blg_scl_join.building_id_acct.notnull(), 'source_detail'] = 'subsidized_model'

        if 'subsidized_units' in blg_scl_join:
            print('total subsidized units of subsidiezed buildings from blg output: {}'.format(blg_scl_join.subsidized_units.sum()))

        blg_scl_join[['parcel_id','building_id', 'building_id_acct',
                      'juris', 'year_built', 'year_acct', 'source', 'source_detail',
                      'residential_units', 
                      'residential_units_acct', 
                      'subsidized_units_acct']].to_csv('blg_data_'+ version_name(i) +'.csv', index=False)
            
        
        blg_scl_subsidized = blg_scl_join.loc[blg_scl_join.building_id_acct.notnull()]

        parcel_cnt = len(blg_scl_subsidized.parcel_id.unique())
        blg_cnt = blg_scl_subsidized.shape[0]
        print('{} subsidized buildings on {} parcels'.format(blg_cnt, parcel_cnt))
        
        print('total res units from acct: {}'.format(blg_scl_subsidized.residential_units_acct.sum()))
        print('total subsidized units from acct: {}'.format(blg_scl_subsidized.subsidized_units_acct.sum()))

        blg_scl_non_subsidized = blg_scl_join.loc[blg_scl_join.building_id_acct.isnull()]
        
        p_all_blg_count = blg_scl_join.groupby('parcel_id')['building_id'].count().reset_index()
        p_all_blg_count.columns = ['parcel_id', 'num_blg']
        
        p_all_blg_units = blg_scl_join.groupby('parcel_id')['residential_units','subsidized_units_acct'].sum().reset_index()
        p_all_blg_units.columns = ['parcel_id', 'res_units', 'subsidized_units']
        
        p_sub_blg_count = blg_scl_subsidized.groupby('parcel_id')['building_id'].count().reset_index()
        p_sub_blg_count.columns = ['parcel_id', 'num_sub_blg']
        
        p_sub_blg_units = blg_scl_subsidized.groupby('parcel_id')['residential_units','subsidized_units_acct'].sum().reset_index()
        p_sub_blg_units.columns = ['parcel_id', 'res_units_subBlg', 'subsidized_units_subBlg']
        
        p_nonsub_blg_count = blg_scl_non_subsidized.groupby('parcel_id')['building_id'].count().reset_index()
        p_nonsub_blg_count.columns = ['parcel_id', 'num_nonsub_blg']
        
        p_nonsub_blg_units = blg_scl_non_subsidized.groupby('parcel_id')['residential_units'].sum().reset_index()
        p_nonsub_blg_units.columns = ['parcel_id','res_units_nonsubBlg']
    
        parcel_summary = p_all_blg_count.merge(p_all_blg_units,
                                               on='parcel_id',
                                               how='left').merge(p_sub_blg_count,
                                                                 on='parcel_id',
                                                                 how='left').merge(p_sub_blg_units,
                                                                                   on='parcel_id',
                                                                                   how='left').merge(p_nonsub_blg_count,
                                                                                                     on='parcel_id',
                                                                                                     how='left').merge(p_nonsub_blg_units,
                                                                                                                       on='parcel_id',
                                                                                                                       how='left')
    
    else:
        print('no subsidized blgs in acctlog')
        
        p_all_blg_count = blg_scl.groupby('parcel_id')['building_id'].count().reset_index()
        p_all_blg_count.columns = ['parcel_id', 'num_blg']
        display(p_all_blg_count.num_blg.sum())
        
        p_all_blg_units = blg_scl.groupby('parcel_id')['residential_units'].sum().reset_index()
        p_all_blg_units.columns = ['parcel_id', 'res_units']
        display(p_all_blg_units[['res_units']].sum())
        
        parcel_summary = p_all_blg_count.merge(p_all_blg_units,
                                               on='parcel_id',
                                               how='left')
        parcel_summary['num_nonsub_blg'] = parcel_summary['num_blg']
        parcel_summary['res_units_nonsubBlg'] = parcel_summary['res_units']
        
        for col_name in ['num_sub_blg', 'res_units_subBlg', 'subsidized_units', 'subsidized_units_subBlg']:
            parcel_summary[col_name] = 0    
        
        if 'subsidized_units' in blg_scl:
            blg_scl_sub = blg_scl.loc[blg_scl.subsidized_units > 0]
            if blg_scl_sub.shape[0] > 0:
                print('blg output has subsidized units: {} buildings, {} units'.format(
                    blg_scl_sub.shape[0],
                    blg_scl_sub.subsidized_units.sum()))


    # add additional fields and export
    print('{} parcels have buildings'.format(parcel_summary.shape[0]))
    
    parcel_summary = parcel_summary.merge(parcel_geo_scl,
                                          left_on='parcel_id',
                                          right_on='PARCEL_ID',
                                          how='outer')
    parcel_summary.fillna(0, inplace=True)
    
    parcel_summary['category'] = np.nan
    parcel_summary.loc[parcel_summary.num_blg == 0, 'category'] = 'no blg'
    parcel_summary.loc[(parcel_summary.num_blg > 0) & (parcel_summary.res_units == 0), 'category'] = 'only non-res blg'
    parcel_summary.loc[(parcel_summary.res_units > 0) & (parcel_summary.num_sub_blg == 0), 'category'] = 'has res_blg but no subsidized'
    parcel_summary.loc[parcel_summary.num_sub_blg > 0, 'category'] = 'has subsidized res_blg'
    
    display(parcel_summary.category.unique())
                
    print('export {} records of parcel_summary for {}'.format(parcel_summary.shape[0], version_name(i)))
    parcel_summary[['parcel_id', 'category', 'num_blg', 'num_sub_blg', 'num_nonsub_blg',
                    'res_units', 'res_units_subBlg', 'res_units_nonsubBlg',
                    'subsidized_units', 'subsidized_units_subBlg']].to_csv('parcel_summary_'+ version_name(i) +'.csv', index=False)
                
    print('')
    #display(blg.head())
    #display(acct.head())

v2.1
4832 subsidized buildings on 4832 parcels
total res units from acct: 57862.0
total subsidized units from acct: 16007.0


C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


422255 parcels have buildings


array(['has res_blg but no subsidized', 'only non-res blg',
       'has subsidized res_blg', 'no blg'], dtype=object)

export 436449 records of parcel_summary for v2.1

v2.2.1 (growth summary updates)
total subsidized units of subsidiezed buildings from blg output: 6406.0
1889 subsidized buildings on 1889 parcels
total res units from acct: 22721.0
total subsidized units from acct: 6406.0
422235 parcels have buildings


...

432714

res_units    998437.0
dtype: float64

422258 parcels have buildings


array(['has res_blg but no subsidized', 'only non-res blg', 'no blg'],
      dtype=object)

export 436449 records of parcel_summary for v2.9

